<a href="https://colab.research.google.com/github/Munya07/NLP/blob/master/1_word_embeddings/Sententence_classification_fast_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install numpy==1.16.1
import re
import itertools
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
!pip install gensim
from gensim.models import FastText
# Original fasttext embeddings from https://fasttext.cc/
from google.colab import drive
drive.mount('/content/drive')
ft = FastText.load_fasttext_format("drive/My Drive/Colab Notebooks/cc.ru.300.bin")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
data= pd.read_excel("drive/My Drive/Colab Notebooks/datasetfinal.xlsx")

In [0]:
import pandas as pd
#WS = pd.read_excel('ur.xlsx')
#WS_np = np.array(WS)
text = data['text']
labels=data['sentiment']

In [0]:
text2 = data['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'@\S+', '', regex=True).replace(r'#\S+', '', regex=True)

In [0]:
import re 
count=[]
for i in range(len(text2)):
    for word in text2[i]:
        res = len(re.findall(r'\w+', text2[i])) 
    count.append(res)
max(count)

27

In [0]:
text2.shape

(3567,)

In [0]:
def clean_str(string):
    """
    Tokenization/string cleaning.
    """
    
    remove_special_chars = re.compile("[^A-Za-z0-9 ]+")
    #This removes any line breaks and replaces them with spaces
    string = string.lower().replace("<br />", " ")
    string=re.sub(remove_special_chars, "", string.lower())
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
   
    string = re.sub("^\s+|\s+$", "", string, flags=re.UNICODE)
    string = " ".join(re.split("\s+", string, flags=re.UNICODE))
    
    
    return string
  
maxlen=27
"""
Pads all sentences to the same length. The length is defined by maxlen.
Returns padded sentences.
"""
padded_sentences = []
for i in range(len(text2)):
  if(len(re.findall(r'\w+', text2[i])) <= maxlen):
      num=maxlen-len(re.findall(r'\w+', text2[i]))
      if num==0:
        print(i)
      f=text2[i]+(" let " * num)
      padded_sentences.append(f)

842


In [0]:
padded_sentences[842]

'Чистая прибыль Сбербанка по РСБУ в январе-июле увеличилась на 3,7% в годовом выражении до 220,3 млрд руб. против 212,4 млрд руб. в январе…'

In [0]:
text2[842]

'Чистая прибыль Сбербанка по РСБУ в январе-июле увеличилась на 3,7% в годовом выражении до 220,3 млрд руб. против 212,4 млрд руб. в январе…'

In [0]:
len(padded_sentences)

3567

In [0]:
padded_sentences[20]

' Лайфхак дня: напишите на стене  и через пол часа там выстроиться очередь из 30 пенсионеров let  let  let  let  let  let  let  let  let  let  let  let '

In [0]:
find_similar_to = 'car'.lower()
ft.similar_by_word(find_similar_to)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cars', 0.7301392555236816),
 ('Car', 0.6899529099464417),
 ('vehicle', 0.643256425857544),
 ('truck', 0.6273175477981567),
 ('automobile', 0.6211560368537903),
 ('driving', 0.6044845581054688),
 ('vehicles', 0.6017204523086548),
 ('rental', 0.6012247800827026),
 ('saloon', 0.5991286039352417),
 ('taxi', 0.5971087217330933)]

In [0]:
len(padded_sentences)

3567

In [0]:
words = []
embedding = []

for i in range(len(padded_sentences)):
  words = []
  f = 0
  j=re.findall(r'\w+', padded_sentences[i])
  for word in re.findall(r'\w+', padded_sentences[i]):
    #print(word)
    # Break the loop if limit exceeds 
    #if i == limit: break
    try :
      vect = ft.wv[word]
    except KeyError:
      print(word)
      vect = ft.wv["none"]
    # 20 other lines
      
    # Getting token 
    words.append(vect)
    f=f+1
    # Appending the vectors 
  embedding.append(words)

мисшн
__
рсг
ЯДом
wp8
о_О


In [0]:
len(embedding[0])

27

In [0]:
d=pd.DataFrame(embedding)

In [0]:
d.shape

(3567, 27)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(embedding, labels, test_size=0.3, random_state=42)

In [0]:
x_train=np.asarray(x_train)
x_test=np.asarray(x_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)
x_train2=x_train.transpose(0,2,1)
x_test2=x_test.transpose(0,2,1)


In [0]:
x_train2 = x_train2.reshape(x_train2.shape[0], 300, 27, 1)
x_test2 = x_test2.reshape(x_test2.shape[0], 300, 27, 1)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

model = Sequential()
model.add(Conv2D(50, kernel_size=(3,7), input_shape=(300,27,1)))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 3)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(256,activation=tf.nn.relu))
model.add(Dense(300,activation=tf.nn.relu))
model.add(Dense(300,activation=tf.nn.relu))
model.add(Dropout(0.3))
model.add(Dense(1,activation=tf.nn.softmax))

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.fit(x_train2, y_train,
          batch_size=60,
          epochs=3,
          verbose=1,
          validation_data=(x_test2, y_test))
score = model.evaluate(x_test2, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 2496 samples, validate on 1071 samples
Epoch 1/3
2496/2496 [==============================] - 35s 14ms/step - loss: 25.3443 - acc: 0.2051 - val_loss: 24.5016 - val_acc: 0.2316
Epoch 2/3
2496/2496 [==============================] - 33s 13ms/step - loss: 25.3443 - acc: 0.2051 - val_loss: 24.5016 - val_acc: 0.2316
Epoch 3/3
2496/2496 [==============================] - 33s 13ms/step - loss: 25.3443 - acc: 0.2051 - val_loss: 24.5016 - val_acc: 0.2316
Test loss: 24.50155572864522
Test accuracy: 0.231559290480213


In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

yhat_probs = model.predict(x_test2, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(x_test2, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)
 
# kappa
kappa = cohen_kappa_score(y_test, yhat_classes)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(y_test, yhat_probs)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)

Accuracy: 0.231559
Precision: 0.231559
Recall: 1.000000
F1 score: 0.376042
Cohens kappa: 0.000000
ROC AUC: 0.500000
[[  0 823]
 [  0 248]]
